<a href="https://colab.research.google.com/github/barbaroja2000/llm/blob/main/Hugging_Face_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Huggingface - Stable Diffusion

In [ ]:
!python -m pip install  -U -q diffusers torch transformers accelerate scipy ftfy

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import transformers
import accelerate
import scipy 
import ftfy

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
#@markdown *Universal negative prompt (Only required for v2 models)*: ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, disfigured, deformed, body out of frame, bad anatomy, watermark, signature, cut off, low contrast, underexposed, overexposed, bad art, beginner, amateur, distorted face

model_id = "runwayml/stable-diffusion-v1-5" #@param ["runwayml/stable-diffusion-v1-5", "stabilityai/stable-diffusion-2-1"] {allow-input: true}
prompt = "two cats grinning at the camera. bamboo forest background japanese woodblock style" #@param {type:"string"}
#not required for v1 models
negative_prompt="photorealistic style, people, out of frame, no cats" #@param {type:"string"}



In [ ]:
#@markdown Lose  `torch_dtype=torch.float16` for full precision
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda:0")

In [ ]:
if model_id == "stabilityai/stable-diffusion-2-1":
  image = pipe(prompt, negative_prompt=negative_prompt).images[0]  
else:
   image = pipe(prompt).images[0]  
image.show()

In [ ]:
#@title Grid
num_cols = 3
num_rows = 3

prompt = [prompt] * num_cols

all_images = []
for i in range(num_rows):
  if model_id == "stabilityai/stable-diffusion-2-1":
    images = pipe(prompt, negative_prompt=negative_prompt).images 
  else:
    images = pipe(prompt).images
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid